In [36]:
import numpy as np

In [53]:
testinput = [
"RRRRIICCFF",
"RRRRIICCCF",
"VVRRRCCFFF",
"VVRCCCJFFF",
"VVVVCJJCFE",
"VVIVCCJJEE",
"VVIIICJJEE",
"MIIIIIJJEE",
"MIIISIJEEE",
"MMMISSJEEE",
]

In [101]:
class Node:
    def __init__(self, posx, posy, plant):
        self.posx = posx 
        self.posy = posy
        self.plant = plant
        self.partners = { self }
        self.needfence = 0
    def __eq__(self, other):
        return self.posx == other.posx and self.posy == other.posy and self.plant == other.plant
    def __hash__(self):
        return hash(str(self.posx) + str(self.posy) + self.plant)
    def getPartners(self):
        return self.partners
    def getPlant(self):
        return self.plant
    def addFence(self):
        self.needfence += 1
    def getFence(self):
        return self.needfence
    def addPartner(self, partner):
        newpartners = partner.partners
        newpartners = newpartners.union(self.partners)
        for p in newpartners:
            p.partners = newpartners

def convertinput(ipt):
    m = {}
    for i, l in enumerate(ipt):
        m[i] = {}
        for ic, c in enumerate(l):
            m[i][ic] = Node(ic, i, c)
            #m[i][ic].partners.add(m[i][ic])
    return m

def connectInput(ipt):
    for y in ipt:
        for x in ipt[y]:
            for d in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                testpos = np.array((y, x)) + np.array(d)
                crop = ipt[y][x].getPlant()
                if 0<= testpos[0] < len(ipt) and 0<= testpos[1] < len(ipt[y]):
                    if crop == ipt[testpos[0]][testpos[1]].getPlant():
                        ipt[y][x].addPartner( ipt[testpos[0]][testpos[1]] )
                    else: ipt[y][x].addFence()
                else: ipt[y][x].addFence()
    return ipt

def removeifExists(ipt, node):
    if node.posy in ipt and node.posx in ipt[node.posy]:
        ipt[node.posy].pop(node.posx)
        if len(ipt[node.posy])==0:
            ipt.pop(node.posy)

def countSides(plot):
    LeftNothing = []
    RightNothing = []
    TopNothing = []
    BottomNothing = []
    for p in plot:
        left, right, top, bottom = [None, None, None, None]
        x, y = p.posx, p.posy
        for p2 in plot:
            x2, y2 = p2.posx, p2.posy
            if y==y2 and x2==x-1: left=p
            if y==y2 and x2==x+1: right=p
            if x==x2 and y2==y-1: top=p
            if x==x2 and y2==y+1: bottom=p
        if not left: LeftNothing.append(p)
        if not right: RightNothing.append(p)
        if not top: TopNothing.append(p)
        if not bottom: BottomNothing.append(p)
    sides = 0
    LeftNothing.sort(key=lambda p: p.posy)
    for y in LeftNothing:
        count = True
        x = y.posx
        for y2 in LeftNothing:
            if x == y2.posx and y.posy - 1 == y2.posy: count=False
        if count: sides+=1
    RightNothing.sort(key=lambda p: p.posy)
    for y in RightNothing:
        count = True
        x = y.posx
        for y2 in RightNothing:
            if x == y2.posx and y.posy - 1 == y2.posy: count=False
        if count: sides+=1
    TopNothing.sort(key=lambda p: p.posx)
    for x in TopNothing:
        count = True
        y = x.posy
        for x2 in TopNothing:
            if y == x2.posy and x.posx - 1 == x2.posx: count=False
        if count: sides+=1
    BottomNothing.sort(key=lambda p: p.posx)
    for x in BottomNothing:
        count = True
        y = x.posy
        for x2 in BottomNothing:
            if y == x2.posy and x.posx - 1 == x2.posx: count=False
        if count: sides+=1
    return sides
    
def findPlot(ipt):
    if len(ipt) == 0: return []
    iptc = ipt.copy()
    plot = set()
    plotroot = None
    for y in iptc:
        for x in iptc[y]:
            #found crop
            plotroot = iptc[y][x]
            break
        break
    # add to plot and remove it from map
    if plotroot:
        fence = 0
        plot.add(plotroot)
        removeifExists(iptc, plotroot)
        for p in plotroot.getPartners():
            fence += p.getFence()
            plot.add(p)
            removeifExists(iptc, p)
        return [ [ plotroot.getPlant(), fence, len(plot), plot ] ] + findPlot(iptc)
            
def findAllPlots(ipt):
    plots = findPlot(ipt)
    totalvalue = 0
    newtotalvalue = 0
    for p in plots:
        if len(p) > 0:
            value = p[1]*p[2]
            newFence = countSides(p[3])
            newprize = newFence*p[2]
            totalvalue += value
            newtotalvalue += newprize
            print("         type: "+p[0], "size: "+str(p[2]), "fence: "+str(p[1]), "prize: "+str(value))
            print("                newfence: " +str(newFence), "newprize: "+str(newprize))
    print("total prize: ", str(totalvalue))
    print("new total prize: ", str(newtotalvalue))
            

    

In [102]:
findAllPlots(connectInput(convertinput(testinput)))

         type: R size: 12 fence: 18 prize: 216
                newfence: 10 newprize: 120
         type: I size: 4 fence: 8 prize: 32
                newfence: 4 newprize: 16
         type: C size: 14 fence: 28 prize: 392
                newfence: 22 newprize: 308
         type: F size: 10 fence: 18 prize: 180
                newfence: 12 newprize: 120
         type: V size: 13 fence: 20 prize: 260
                newfence: 10 newprize: 130
         type: J size: 11 fence: 20 prize: 220
                newfence: 12 newprize: 132
         type: C size: 1 fence: 4 prize: 4
                newfence: 4 newprize: 4
         type: E size: 13 fence: 18 prize: 234
                newfence: 8 newprize: 104
         type: I size: 14 fence: 22 prize: 308
                newfence: 16 newprize: 224
         type: M size: 5 fence: 12 prize: 60
                newfence: 6 newprize: 30
         type: S size: 3 fence: 8 prize: 24
                newfence: 6 newprize: 18
total prize:  1930
new total pri

In [103]:
ipt = [l.strip("\n") for l in open("input.txt")]

In [104]:
findAllPlots(connectInput(convertinput(ipt)))

         type: L size: 116 fence: 94 prize: 10904
                newfence: 46 newprize: 5336
         type: F size: 93 fence: 70 prize: 6510
                newfence: 44 newprize: 4092
         type: I size: 54 fence: 52 prize: 2808
                newfence: 36 newprize: 1944
         type: V size: 83 fence: 60 prize: 4980
                newfence: 42 newprize: 3486
         type: Z size: 18 fence: 28 prize: 504
                newfence: 18 newprize: 324
         type: E size: 78 fence: 62 prize: 4836
                newfence: 34 newprize: 2652
         type: N size: 4 fence: 10 prize: 40
                newfence: 6 newprize: 24
         type: K size: 82 fence: 60 prize: 4920
                newfence: 36 newprize: 2952
         type: A size: 2 fence: 6 prize: 12
                newfence: 4 newprize: 8
         type: R size: 37 fence: 34 prize: 1258
                newfence: 16 newprize: 592
         type: P size: 148 fence: 82 prize: 12136
                newfence: 46 newprize: 6808
 